In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from MongoCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "pickle123"
shelter = AnimalShelter(username, password)



# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

# # Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)
# print(df.head())

10003
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks', 'age'],
      dtype='object')
   rec_num age_upon_outcome animal_id animal_type  \
0      8.0           1 year   A736551         Dog   
1      4.0         7 months   A733653         Cat   
2      9.0          3 years   A720214         Dog   
3     10.0         3 months   A664290         Cat   
4     11.0           1 year   A721199         Dog   

                                      breed       color date_of_birth  \
0  Labrador Retriever/Australian Cattle Dog       Black    2015-10-12   
1                               Siamese Mix  Seal Point    2016-01-25   
2                    Labrador Retriever Mix   Red/White    2013-02-04   
3                    Domestic Shorthair Mix      Tortie    2013-09-01   
4       

In [7]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #Use div with display flex to align items horizontally and center
    html.Div([
        html.Img(src='assets/dog_shelter_logo.png', style={'height':'100px', 'width':'100px'}), 
        html.H1('SNHU CS-340 Dashboard', style={'marginLeft': '20px'}),
    ], style={'display': 'flex', 'alignItems':'center', 'justifyContent':'center'}),
    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),

        # Enabling single row selection
        row_selectable='single',

        # Enabling sorting
        sort_action='native',

        # Enabling filtering
        filter_action='native',

        # Enabling pagination
        page_action='native',
        page_current=0,  # page number that the user is currently on
        page_size=15,  # number of rows visible per page

        # Conditional formatting can be added here based on your requirements

        # Additional optional features for enhancing user interaction
        style_table={'overflowX': 'auto'},  # Horizontal scroll
        style_cell={
            'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },  # Ensuring text in each cell is visible without altering the table layout
        style_header={
            'backgroundColor': 'white',
            'fontWeight': 'bold'
        },  # Styling for the header to differentiate from row entries
),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',
            )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return [] #Return an empty list if none
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, selected_rows):
    #default center and zoom
    center = [20,0]
    zoom = 2
    
    if viewData is None or not selected_rows:
        #Return default if no row is selected
        return [dl.Map(center=center, zoom=zoom, children=[dl.TileLayer()], style={'width':'1000px', 'height':'500px'})]        


    # This conversion step is necessary to access the selected row correctly
    rows = pd.DataFrame.from_records(viewData)  # Convert viewData to a DataFrame
    
    # Now, you can access the selected_row as shown
    if selected_rows and viewData and selected_rows[0] < len(rows):
        
        selected_row = rows.iloc[selected_rows[0]] #if row is selected row
        latitude = selected_row['location_lat']
        longitude = selected_row['location_long']
        center = [latitude, longitude] #center on selected row
        zoom = 10 #Zoom in to location
        
        #create a marker
        marker = dl.Marker(position=center, children=[
            dl.Tooltip("Selected Location"),
            dl.Popup("This is the selected location.")
        ])
        
        #Updating the map's children to include the new marker
        return [dl.Map(center=center, zoom=zoom, children=[
            dl.TileLayer(),
            marker
        ], style={'width': '1000px', 'height': '500px'})]
    


app.run_server(debug=True)

Dash app running on http://127.0.0.1:25127/
